# 연관규칙_연습

### Step 0 mlxtend 설치
- 설치 안했으면 실행해서 설치 
- 설치 했으면 pass

In [1]:
!pip install mlxtend

### Step 0 패키지 불러오기

In [4]:
# 데이터 구성 : Seires, DataFrame
import pandas as pd 

# 행렬 연산 
import numpy as np

# 데이터 전처리 : 항목 값에 대한 index 생성
from mlxtend.preprocessing import TransactionEncoder

# 지지도 계산
from mlxtend.frequent_patterns import apriori

# 연관 규칙 
from mlxtend.frequent_patterns import association_rules

### Step 1 데이터 구성하기  

- 데이터 생성

In [6]:
df_raw = [["소주","콜라","맥주"],
        ["소주","콜라","와인"],
        ["소주","주스"],
        ["콜라","맥주"],
        ["소주","콜라","맥주","와인"],
        ["주스"]]
    
df_raw

[['소주', '콜라', '맥주'],
 ['소주', '콜라', '와인'],
 ['소주', '주스'],
 ['콜라', '맥주'],
 ['소주', '콜라', '맥주', '와인'],
 ['주스']]

### Step 1 데이터 구성하기 

- 데이터 전처리

In [9]:
# items 중 index 0번이 갖고 있으면 Ture, 아니면 False
enc = TransactionEncoder()
df_raw_enc = enc.fit_transform(X = df_raw)

# 위의 함수는 데이터를 행렬ㄹ의 형태로 반환 pandas의 DataFrame의 형태로 반환
df_asso = pd.DataFrame(df_raw_enc, columns = enc.columns_)
df_asso.head()

,맥주,소주,와인,주스,콜라
0,True,True,False,False,True
1,False,True,True,False,True
2,False,True,False,True,False
3,True,False,False,False,True
4,True,True,True,False,True


### Step 2 라이브러리 불러오기

- 규칙 척도 임계값(지지도 50% 이상)

In [12]:
# 최소 지지도
v_min_sup = 0.5

# 지지도 계산
df_freq = apriori(df_asso, min_support = v_min_sup, use_colnames = True)
df_freq.round(3)

,support,itemsets
0,0.500,(맥주)
1,0.667,(소주)
2,0.667,(콜라)
3,0.500,"(콜라, 맥주)"
4,0.500,"(콜라, 소주)"


- 맥주를 구매한 고객 : 0.5  
- 소주를 구매한 고객 : 0.667   
- 맥주와 콜라를 동시에 구매한 고객 : 0.5

### Step 2 라이브러리 불러오기



- 지지도, 신뢰도, 향상도 산출 (기준: 지지도(support) / 신뢰도(confidence) / 향상도(lift). 임계값:기준 metric의 최소값 

In [16]:
# 기준 = 신뢰도, 최소 신뢰도 = 0.5 
df_asso_rule = association_rules(df_freq, metric = "confidence", min_threshold = 0.5)
df_asso_rule.round(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(콜라),(맥주),0.667,0.500,0.5,0.75,1.500,0.167,2.000
1,(맥주),(콜라),0.500,0.667,0.5,1.00,1.500,0.167,inf
2,(콜라),(소주),0.667,0.667,0.5,0.75,1.125,0.056,1.333
3,(소주),(콜라),0.667,0.667,0.5,0.75,1.125,0.056,1.333


### Step 3 결과 해석 및 결론 도출

> [(맥주),(콜라)] Confidence = 1.0 : 맥주를 구매한 고객은 콜라도 반드시 구매 -> 맥주를 구매한 고객에게 콜라를 추천

> [(콜라),(맥주)] Confidence = 0.75 : 콜라를 구매한 고객 중 맥주를 고른 고객은 0.75   

> [(콜라),(소주)] Lift = 1.125  
> [(콜라),(맥주)] Lift = 1.5 : 콜라를 구매한 고객들에게는 소주보다 맥주를 추천